# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    print("API key looks good")
else:
    print("Check your API key — see troubleshooting notebook.")

openai = OpenAI()
openrouter = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=api_key)

In [ ]:
system_prompt = """
You are a computer science expert with deep knowledge of the subject and a passion for teaching.
You can explain complex concepts in a way that is easy to understand, breaking them down into smaller, understandable pieces, provide explample usecases and citint a maximum of 3 sources (e.g Book,(Author), Website, etc.)  while maintaining the overall context and connections between ideas and attaching clickable links to the sources.

"""

In [ ]:
MODEL_GPT = "gpt-4o-mini"
MODEL_GEMINI = "google/gemini-2.5-flash"
MODEL_GROK_4_FAST = "x-ai/grok-4.1-fast"
MODEL_CLAUDE_SONNET_3_5 = "anthropic/claude-3.5-sonnet"

MODELS = [
    {"label": "GPT-4o mini", "model": MODEL_GPT, "client": openrouter},
    {"label": "Gemini 3.1", "model": MODEL_GEMINI, "client": openrouter},
    {"label": "Grok 4", "model": MODEL_GROK_4_FAST, "client": openrouter},
    {"label": "Claude 3.5 Sonnet", "model": MODEL_CLAUDE_SONNET_3_5, "client": openrouter},
]
MODEL_BY_LABEL = {m["label"]: m for m in MODELS}
DROPDOWN_CHOICES = [m["label"] for m in MODELS]

In [ ]:
def expert_tutor(messages: list[dict], model: str = MODEL_GPT, client: OpenAI = openai):
    print(f"Researching question to provide a source backed explanation {model}")
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
def history_to_messages(history, new_user_message: str) -> list[dict]:
    messages = [{"role": "system", "content": system_prompt}]

    for turn in (history or []):
        if isinstance(turn, dict):
            messages.append({"role": turn["role"], "content": turn.get("content", "") or ""})
        else:
            user_msg, assistant_msg = turn[0], turn[1]
            if user_msg:
                messages.append({"role": "user", "content": user_msg})
            if assistant_msg:
                messages.append({"role": "assistant", "content": assistant_msg})

    messages.append({"role": "user", "content": new_user_message})
    return messages

In [ ]:
def run_tutor(message: str, history, model_choice: str):
    if isinstance(message, list) and isinstance(history, str):
        message, history = history, message
    if not message or not message.strip():
        return "Please enter a message."

    messages = history_to_messages(history, message)
    m = MODEL_BY_LABEL[model_choice]
    yield from expert_tutor(messages, model=m["model"], client=m["client"])

In [ ]:
with gr.Blocks(title="Expert Tutor") as demo:
    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=DROPDOWN_CHOICES,
            label="Model",
            value=DROPDOWN_CHOICES[0],
        )
    gr.ChatInterface(fn=run_tutor, additional_inputs=[model_dropdown])

demo.launch(inbrowser=True)